In [1]:
# import initializer
import kernel_init

In [2]:
# import packages
import datetime as dt
from django.contrib.auth.models import User
from django.db import transaction
from django.core.exceptions import MultipleObjectsReturned, ObjectDoesNotExist
import json
import os
import pandas as pd
import pytz as tz
import sys
from tqdm import tqdm
import pathlib
import numpy as np

In [3]:
# import local files
from purchasing.models import *
from purchasing.serializers import *

In [4]:
# set up file path and read into dataframe
data_path = r'C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works\Purchasing\_archive\merged_purchase_records.csv'

# read main purchase list into dataframe
df = pd.read_csv(data_path)
df = df.where(pd.notnull(df), None)
print(df.head(10))
rows, cols = df.shape
print("DF Shape ---> rows: ", rows, "  cols: ", cols)

print(df.columns.values)

        id  requested requested_by_id   modified modified_by_id vendor_id  \
0  1000002  8/30/2019          hsmith  8/30/2019         hsmith    Amazon   
1  1000006  8/30/2019          acable  8/30/2019         acable  Thorlabs   
2  1000007  8/30/2019          acable  8/30/2019         acable  Thorlabs   
3  1000008  8/30/2019          acable  8/30/2019         acable  Thorlabs   
4  1000009  8/30/2019         afisher  8/30/2019        afisher    Airgas   
5  1000010  8/30/2019         afisher  8/30/2019        afisher       VWR   
6  1000011  8/30/2019         afisher  8/30/2019        afisher       VWR   
7  1000012  8/30/2019         afisher  8/30/2019        afisher    Amazon   
8  1000013  8/30/2019         afisher  8/30/2019        afisher       VWR   
9  1000014  8/30/2019         afisher  8/30/2019        afisher       VWR   

                                         description  product_no package_size  \
0                                       X-acto knife       X3311       

In [5]:
users = User.objects.all()
all_vendors = Vendor.objects.all()
all_categories = Category.objects.all()
all_groups = Group.objects.all()
all_status = ItemStatus.objects.all()
all_approvals = ApprovalStatus.objects.all()
all_projects = Project.objects.all()


In [6]:
# validate FK objects
df_vendors = [str(s) for s in df['vendor_id'].dropna().unique()]
df_vendors.sort()
print(f"Found {len(df_vendors)} vendors.")

Found 278 vendors.


In [7]:
def get_vendor_code(v):
    try:
        v_obj = all_vendors.get(name__icontains=v)
        return v_obj.vendor_code
    except MultipleObjectsReturned as ex:
        try:
            v_obj = all_vendors.get(name=v)
            return v_obj.vendor_code
        except:
            return None
    except Exception as ex:
        return None

In [8]:
# configure settings for importing data
batch_size = 12
remainder = rows % batch_size
n_iters = int(rows/batch_size)
print("batch size: ", batch_size, "  n_iters: ", n_iters, " remainder: ", remainder)

start = 0
end = batch_size

vendor_errors = []

logfile = open('logfile.txt', 'w')
logfile.write("Beginning sync...\n")
logfile.close()

for i in tqdm(range(n_iters)):
    batch = []
    df_ids = []
    with open('logfile.txt', 'a') as logfile:
        for j in range(start, end):
            batch.append({
                'description': str(df['description'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'requested': df['requested'][j],      
                'modified': df['modified'][j],
                'product_no': str(df['product_no'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'package_size': str(df['package_size'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'moq': str(df['moq'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'item_cost': df['item_cost'][j],
                'qty': df['qty'][j],
                'total_cost': df['total_cost'][j],
                'product_link': df['product_link'][j],
                'qty_received': int(df['qty_received'][j]),
                'notes': str(df['notes'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'po_no': str(df['po_no'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'quote_no': str(df['quote_no'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'modified_by': str(df['modified_by_id'][j]),
                'requested_by': str(df['requested_by_id'][j]),
                'approved': str(df['approved_id'][j]),
                'status': str(df['status_id'][j]),
                'group_id': str(df['group_id'][j]),
                'category': all_categories.get(description=str(df['category_id'][j])).category_code if df['category_id'][j] is not None else None,
                'project': all_projects.get(name=str(df['project_id'][j])).project_no if df['project_id'][j] is not None else None,
                'vendor': get_vendor_code(str(df['vendor_id'][j])) if df['vendor_id'][j] is not None else None,
            })

    with open('logfile.txt', 'a') as logfile:
        batch_serializer = PurchaseSerializer(data=batch, many=True)
        if batch_serializer.is_valid():
            try:
                with transaction.atomic():
                    batch_serializer.save()
            except Exception as ex:
                logfile.write(f"ERROR --->  Batch: {i+1}  ID: {df['id'][j]} Upload failed. {str(ex)} \n")
        else:
            logfile.write("[SERIALIZER ERRORS] ---> " + str(batch_serializer.errors) + '\n')

        start += batch_size
        end += batch_size

        logfile.write(f"Batch {i+1} of {n_iters} completed..\n")


batch size:  12   n_iters:  419  remainder:  0


100%|██████████| 419/419 [44:01<00:00,  6.31s/it] 


In [38]:
# add missing purchases one by one and get error messages
err_df = pd.read_csv('./record_errors.csv')
err_df = err_df.where(pd.notnull(df), None)

with open('err_log.txt', 'w') as err:
    for j in tqdm(range(len(err_df['id'].values))):
        try:
            new_purchase = PurchaseSerializer(data={
                'description': str(err_df['description'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'requested': err_df['requested'][j],      
                'modified': err_df['modified'][j],
                'product_no': str(err_df['product_no'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'package_size': str(err_df['package_size'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'moq': str(err_df['moq'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'item_cost': err_df['item_cost'][j],
                'qty': err_df['qty'][j],
                'total_cost': err_df['total_cost'][j],
                'product_link': err_df['product_link'][j],
                'qty_received': int(df['qty_received'][j]),
                'notes': str(err_df['notes'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'po_no': str(err_df['po_no'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'quote_no': str(err_df['quote_no'][j]).strip().replace(',', ' ').replace('\n', ' '),
                'modified_by': User.objects.get(username=str(err_df['modified_by_id'][j])),
                'requested_by': User.objects.get(username=str(err_df['requested_by_id'][j])),
                'approved': str(err_df['approved_id'][j]),
                'status': str(err_df['status_id'][j]),
                'group_id': str(err_df['group_id'][j]),
                'category': all_categories.get(description=str(err_df['category_id'][j])).category_code if err_df['category_id'][j] is not None else None,
                'project': all_projects.get(name=str(err_df['project_id'][j])).project_no if err_df['project_id'][j] is not None else None,
                'vendor': get_vendor_code(str(err_df['vendor_id'][j])) if err_df['vendor_id'][j] is not None else None,
            })
            if new_purchase.is_valid():
                new_purchase.save()
            else:
                err.write(str(err_df['id'][j]) + " ---> " + str(new_purchase.errors) + '\n')
        except Exception as ex:
            err.write(str(err_df['id'][j]) + " ---> " + str(ex) + '\n')

100%|██████████| 1/1 [00:00<00:00,  2.38it/s]


In [31]:
from datetime import datetime
from django.utils.timezone import make_aware

requested_dates = [datetime.strptime(dt, "%m/%d/%Y")
                    for dt in df['requested'].values]
requested_dates = [make_aware(t) for t in requested_dates]

with open('record_errors.csv', 'w', encoding='utf-8') as err:
    for i, d in enumerate(tqdm(df['description'].values)):
        try:
            d_obj = Purchase.objects.get(description=d)
            d_obj.requested = requested_dates[i]
            d_obj.save()
        except MultipleObjectsReturned:
            pass
        except Exception as ex:
            err.write(str(ex) + '\n')


  1%|          | 33/5028 [00:02<07:19, 11.38it/s]


KeyboardInterrupt: 

In [35]:
dupes_df = pd.read_csv(r'./duplicate_date_fixes.csv')
print(dupes_df.columns.values)
dupes_desc = [str(s) for s in dupes_df['description'].values]
print(len(dupes_desc))
dupes_desc.sort()

found = 0
for dupe in dupes_desc:
    rows = df.loc[df['description'] == dupe].values
    records = Purchase.objects.filter(description=dupe)
    i = min(len(rows), len(records))
    ## if len(rows) > 0:
    ##     for j in range(i):
    ##         records[j].requested = make_aware(datetime.strptime(rows[j][1], "%m/%d/%Y"))
    ##         records[j].save()
    if len(rows) != len(records):
        print(dupe, len(rows), len(records))


['description' 'product_no' '(No column name)']
344
1 micron Pall Emflon filter 0 3
12mm EFL f/1.4 for 2/3" C-Mount Format Cameras with Lock 1 2
30mm Cage Right-angle kinematic OAP Mirror Mount 3 2
5 Port Switch 2 3
50 mm Square Construction Rail 20" Long 1/4"-20 Taps 1 2
50mm EFL F/2.8 for 2/3" C-Mount Format Cameras with Lock 1 2
6.5X Zoom Lens 1 3
8-32 Cap Screw Kit 1 2
AZ 340 Developer 0 4
Adapter with External SM1 Threads and Internal C-Mount Threads 9.1mm Spacer 1 2
C-Mount Adapter 0 2
Complete Micropipette Kit: 4 Pipettors (0.5-10μl 10-100μl 100-1000μl 1000-5000μl); Pipette Stand; 3 Racks of 96 Sterile Pipette Tips 0 2
DB15 Male and Plastic Hoods 0 2
Double Sided PCB Board Prototype Kit 0 2
Foam Swab Lint Free Cleaning Swabsticks for Camera Optical Lens Electronics Detailing Small Hole 0 2
InGaAs Switchable Gain Detector on PCB 800 - 1700 nm 11 MHz 3.14 mm2 0 2
InGaAs Switchable Gain Detector on PCB 900 - 1700 nm 13 MHz 0.8 mm2 0 2
Infrasil Windows 5 4
MOTORplate 1 2
Optical Cho

In [8]:
n_rows, n_cols = df.shape
purchases = Purchase.objects.filter(group__isnull=True).order_by('requested')

for row in tqdm(range(n_rows)):
    try:
        p_objs = purchases.filter(description=df['description'][row])
        if len(p_objs) > 0:
            p_obj = p_objs[0]
            p_obj.group = all_groups.get(group_code=df['group_id'][row])
            p_obj.save()
            purchases = Purchase.objects.filter(group__isnull=True).order_by('requested')
        #print(p_obj.group, df['group_id'][row])
        #print(f"[LOG] ---> found object {p_obj}")

    except MultipleObjectsReturned as ex:
        #try:
            #v_obj = all_vendors.get(description=df['description'][row])
            #print(f"[LOG] ---> found object {p_obj}")
        #except:
        print(f"[ERR] {ex} ---> {df['description'][row]}")
    except Exception as ex:
        print(f"[ERR] {ex} ---> {df['description'][row]}")

100%|██████████| 5028/5028 [04:11<00:00, 20.02it/s]
